# RTML lab1 report - st121413

Your lab report should have the following sections:

1. Introduction: the background and goals of the lab
2. Methods: what you did, what parameters you tried, and so on
3. Results: what were the results
4. Conclusion: what did you learn from the lab, and what might be the next steps

In the results section, be sure to show training and validation loss as a function of training epochs. You'll also want to show results on a separate test set and give some analysis of the errors the classifier makes on the test set.

##  1. Introduction

The goal of this lab is to set up our machine so that it can connect to Puffer via SSH and Docker inside it.

Then, we try to test whether our docker can run PyTorch on GPUs using AlexNet.

Then, bragging out our accuracy on Piazza.

# 2. Method
Well, the connection to Puffer/Guppy is done successfully and th evidence it shown in the picture.

![evidence](screenshot/ssh-vscode.png)

I also have git for sotring and syncing my data.
[https://github.com/akraradets/2021JanRTML]

In lab 1 folder, there are 4 files.

1. lab1 - Assignment.ipynb
 - is the file I use for testing the code. (For developing and debuging)
2. lab1 - Assignment.py
 - is the file I use for training. I copied the code from .ipynb and run this file using nohup so I can turn off my computer
3. lab1 - In lab.ipynb
 - is the file develop during the lab session
4. lab1 - report.ipynb
 - is this file

## 2.1 Training AlexNet

At first, I tried not to do a lot since I thought we suppose to only test out our environment. Later on, Prof. Matt and one of our classmates start bragging about their accuracy. Therefore, I felt like I need to do more.

Before anything else, I am curious about Multiple GPU. So I did some research on how to do it on Pytorch website. [https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html]
I did try it with various configuration because it very picky about parameters I use. (num_worker, number of GPU)
There is one configuration that seems to work (I think it is a 2 GPUs setup), but the speed of training is lower significantly. I did 'nvidia-smi' and found that one of the GPU run and another one just sit around. It could be a starvation problem.

I am about to write my first report then the bragging started on the Piazza plus a tensorboard.
At the same time, Puffer decided to stop working. All GPUs are occupied (with 0 usage).

My next move is to move to Guppy and I did post my DockerFile on Piazza. It takes me about one day to migrate.

Now, about training the AlexNet. [https://analyticsindiamag.com/implementing-alexnet-using-pytorch-as-a-transfer-learning-model-in-multi-class-classification/]

### 2.1.1 CIFAR-10

Honestly, I did not feel like using any augmentation but Prof. Matt put it in the Hint which force me to do so.

In [ ]:
# 1. Load CIFAR10
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 5. Use some data augmentation for the training set -- RandomCrop(224) and RandomHorizontalFlip() seem to help.
# 6. Normalize the images to the [-1,1] range using Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) if you're training from scratch, or if you're fine tuning the ImageNet weights, you should probably use the magic normalizer of Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), though this can give pixel values outside the [-1,1] range.
preprocess = transforms.Compose([
    transforms.Resize(256),
    # transforms.CenterCrop(224),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 1. Use the torchvision CIFAR10 dataset to download the data and load images into memory
# 2. Use PyTorch Dataloaders to shuffle the data and break them into batches during training
# 3. Use a batch size of 200 or less so that you're not using more than 3 GB of GPU RAM (save some for others!)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=preprocess)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=200, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=preprocess)
testloader = torch.utils.data.DataLoader(testset, batch_size=200, shuffle=False, num_workers=0)

classes = np.array(('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'))

### 2.1.2 The AlexNet
#### 2.1.2.A Attemp A

Eventhough, the website suggest me to change the dense layer 4 to (4096,1024), I decided to keep it the same for the originality of AlexNet design. I only changed the last layer to output 10 classes which is (4096,10)

One more notable mention is that I use lr=0.001 which it took me 300 epochs and still not converged. **sad face**

In [ ]:
# 2. Load AlexNet
# https://github.com/pytorch/vision/releases
# The newest version isa 0.8.2 and load empty model
model = torch.hub.load('pytorch/vision:v0.8.2', 'alexnet', pretrained=False)
print(model.eval())

# https://analyticsindiamag.com/implementing-alexnet-using-pytorch-as-a-transfer-learning-model-in-multi-class-classification/
# Updating the second classifier
# model.classifier[4] = nn.Linear(4096,4096)
# Updating the third and the last classifier that is the output layer of the network. Make sure to have 10 output nodes if we are going to get 10 class labels through our model.
model.classifier[6] = nn.Linear(4096,10)
print(model.eval())

In [ ]:
#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
# 4. For the optimizer, use stochastic gradient descent with a learning rate around 0.01 or 0.02 to start with. I couldn't get the Adam optimizer working in this experiment at all.
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#### 2.1.2.B Attemp B

My second attempt of trying to beat Nattawach and Prof. Matt is to blindly add more node in the dense layer.

Since I had no idea what I am doing, I just copy the pattern of the AlexNet in the Classifier and repeat it.
The pattern is

- Dropout
- Linear
- ReLU



This time, I changed the lr to 0.01 becuase I could not afford to wait 12 hours again.

In [ ]:
# 2. Load AlexNet
# https://github.com/pytorch/vision/releases
# The newest version isa 0.8.2 and load empty model
model = torch.hub.load('pytorch/vision:v0.8.2', 'alexnet', pretrained=False)
print(model.eval())

# https://analyticsindiamag.com/implementing-alexnet-using-pytorch-as-a-transfer-learning-model-in-multi-class-classification/
# Updating the second classifier
model.classifier[4] = nn.Linear(4096,4096)
# Updating the third and the last classifier that is the output layer of the network. Make sure to have 10 output nodes if we are going to get 10 class labels through our model.
model.classifier[6] = nn.Dropout(p=0.5, inplace=False)
model.classifier.add_module('7', nn.Linear(4096,4096))
model.classifier.add_module('8', nn.ReLU(inplace=True))
model.classifier.add_module('9', nn.Dropout(p=0.5, inplace=False))
model.classifier.add_module('10', nn.Linear(4096,4096))
model.classifier.add_module('11', nn.ReLU(inplace=True))
model.classifier.add_module('12', nn.Linear(4096,10))
print(model.eval())

In [ ]:
#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
# 4. For the optimizer, use stochastic gradient descent with a learning rate around 0.01 or 0.02 to start with. I couldn't get the Adam optimizer working in this experiment at all.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 3. Results
## 3.1 Attempt A
![AttempA Result](screenshot/attemptA.png)

My Epoch start with 0 so 299 is round 300.

Unfortunately, I forgot to log Loss/test because I thought accuracy is enough.

This is the last 10 lines from the "nohup.out"

Epoch: number| Loss:{Train Loss} {Avg Train Loss}|Test:{Accuracy Test}|Train:{Accuracy Train}

Epoch:291|Loss:13.638674892485142 0.054554699569940565|Test:0.8549|Train:0.98042

Epoch:292|Loss:12.40823491383344 0.04963293965533376|Test:0.8555|Train:0.98188

Epoch:293|Loss:13.680912331677973 0.05472364932671189|Test:0.8578|Train:0.98434

Epoch:294|Loss:13.622858950868249 0.054491435803472994|Test:0.8616|Train:0.98384

Epoch:295|Loss:13.00801098253578 0.052032043930143115|Test:0.8525|Train:0.98048

Epoch:296|Loss:13.26588236540556 0.05306352946162224|Test:0.8561|Train:0.98378

Epoch:297|Loss:12.807034968398511 0.05122813987359404|Test:0.8581|Train:0.98288

Epoch:298|Loss:12.895507446490228 0.051582029785960916|Test:0.856|Train:0.98278

Epoch:299|Loss:13.561798548325896 0.05424719419330359|Test:0.8532|Train:0.98034

Finished Training of AlexNet


My navie AlexNet achived 85% on test set and nearly (also scary) 100% on Training set.
In addition, because of a learning rate of 0.001, it took 300 epochs and 12h 26m to reach this.

## 3.2 Attempt B
![AttempA Result](screenshot/attemptB.png)

This the last 10 logs of AttemptB (which include TestLoss)

Epoch: number| Loss:{Train Loss} {Avg Train Loss}|Test:{Test Loss}|Test:{Accuracy Test}|Train:{Accuracy Train}

Epoch:32|Loss:21.91974487528205 0.0876789795011282|Test:0.6595877069234848|Test:0.8392|Train:0.9764

Epoch:33|Loss:20.927316272631288 0.08370926509052515|Test:0.6978633767366409|Test:0.8338|Train:0.97236

Epoch:34|Loss:19.337717892602086 0.07735087157040835|Test:0.6463613945245743|Test:0.8384|Train:0.97818

Epoch:35|Loss:18.0199269708246 0.0720797078832984|Test:0.6774795478582383|Test:0.839|Train:0.97708

Epoch:36|Loss:16.214105795137584 0.06485642318055033|Test:0.6600141668319702|Test:0.8397|Train:0.98026

Epoch:37|Loss:15.892928196117282 0.06357171278446913|Test:0.6798667490482331|Test:0.8362|Train:0.9806

Epoch:38|Loss:14.923890792764723 0.059695563171058896|Test:0.6922888559103012|Test:0.8419|Train:0.98198

Epoch:39|Loss:12.694255939684808 0.050777023758739236|Test:0.738852990269661|Test:0.8432|Train:0.9852

Epoch:40|Loss:12.195616493932903 0.04878246597573161|Test:0.6651264733076095|Test:0.8454|Train:0.98742

Epoch:41|Loss:12.237079987302423 0.04894831994920969|Test:0.7097285014390945|Test:0.8364|Train:0.9804

I stoped the training at epoch 41 because it does not seem to improve any this but only reduce the Test Loss (the best loss is 0.5)

## 3.3 Evaluation

For evaluation of attemptA

*Accuracy of the network on the 10000 test images: 88 %*

- Accuracy of plane : 90 %
- Accuracy of   car : 100 %
- Accuracy of  bird : 89 %
- Accuracy of   cat : 65 %
- Accuracy of  deer : 92 %
- Accuracy of   dog : 86 %
- Accuracy of  frog : 86 %
- Accuracy of horse : 93 %
- Accuracy of  ship : 92 %
- Accuracy of truck : 100 %

For evaluation of attemptB

*Accuracy of the network on the 10000 test images: 84 %*

- Accuracy of plane : 81 %
- Accuracy of   car : 94 %
- Accuracy of  bird : 84 %
- Accuracy of   cat : 73 %
- Accuracy of  deer : 88 %
- Accuracy of   dog : 80 %
- Accuracy of  frog : 72 %
- Accuracy of horse : 93 %
- Accuracy of  ship : 96 %
- Accuracy of truck : 100 %

# 4. Conclusion

I used to believe that the more node you have the more capable your model is. This does not seem to hold up in my experiment. All in all, now I know there is more than just having a big model. Thank you to this course, now I am obsessed with models.